In [20]:
import requests
import json
import pandas as pd

from collections import Counter
import numpy as np
import plotly.express as px
import plotly as p

In [4]:
def api_func(year):
    api_URL= f"https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/migr_asydcfsta?format=JSON&time={year}&unit=PER&citizen=LI&citizen=UK_OCT&citizen=BA&citizen=ME&citizen=MD&citizen=MK&citizen=GE&citizen=AL&citizen=RS&citizen=TR&citizen=UA&citizen=XK&citizen=AD&citizen=BY&citizen=VA&citizen=MC&citizen=RU&citizen=SM&citizen=AO&citizen=CM&citizen=CF&citizen=TD&citizen=CG&citizen=CD&citizen=GQ&citizen=GA&citizen=ST&citizen=BI&citizen=KM&citizen=DJ&citizen=ER&citizen=ET&citizen=KE&citizen=MG&citizen=MW&citizen=MU&citizen=MZ&citizen=RW&citizen=SC&citizen=SO&citizen=UG&citizen=TZ&citizen=ZM&citizen=ZW&citizen=DZ&citizen=EG&citizen=LY&citizen=MA&citizen=SS&citizen=SD&citizen=TN&citizen=EH&citizen=BW&citizen=LS&citizen=NA&citizen=ZA&citizen=SZ&citizen=BJ&citizen=BF&citizen=CV&citizen=CI&citizen=GM&citizen=GH&citizen=GN&citizen=GW&citizen=LR&citizen=ML&citizen=MR&citizen=NE&citizen=NG&citizen=SN&citizen=SL&citizen=TG&citizen=AG&citizen=BS&citizen=BB&citizen=CU&citizen=DM&citizen=DO&citizen=GD&citizen=HT&citizen=JM&citizen=KN&citizen=LC&citizen=VC&citizen=TT&citizen=BZ&citizen=CR&citizen=SV&citizen=GT&citizen=HN&citizen=MX&citizen=NI&citizen=PA&citizen=AR&citizen=BO&citizen=BR&citizen=CL&citizen=CO&citizen=EC&citizen=GY&citizen=PY&citizen=PE&citizen=SR&citizen=UY&citizen=VE&citizen=KZ&citizen=KG&citizen=TJ&citizen=TM&citizen=UZ&citizen=CN&citizen=JP&citizen=MN&citizen=KP&citizen=KR&citizen=TW&citizen=AF&citizen=BD&citizen=BT&citizen=IN&citizen=IR&citizen=MV&citizen=NP&citizen=PK&citizen=LK&citizen=BN&citizen=KH&citizen=ID&citizen=LA&citizen=MY&citizen=MM&citizen=PH&citizen=SG&citizen=TH&citizen=TL&citizen=VN&citizen=AM&citizen=AZ&citizen=BH&citizen=IQ&citizen=IL&citizen=JO&citizen=KW&citizen=LB&citizen=PS&citizen=OM&citizen=QA&citizen=SA&citizen=SY&citizen=AE&citizen=YE&citizen=AU&citizen=NZ&citizen=FJ&citizen=PG&citizen=SB&citizen=VU&citizen=KI&citizen=MH&citizen=FM&citizen=NR&citizen=PW&citizen=CK&citizen=WS&citizen=TO&citizen=TV&citizen=RNC&citizen=STLS&citizen=TOTAL&citizen=UNK&sex=T&sex=M&sex=F&sex=UNK&age=TOTAL&age=Y_LT14&age=Y14-17&age=Y_LT18&age=Y18-34&age=Y35-64&age=Y_GE65&age=UNK&decision=TOTAL&decision=TOTAL_POS&decision=REJECTED&lang=en"
    response = requests.get(api_URL)
    data = response.json()

    dimensions = data['dimension']
    values = data['value']


    rows = []


    dim_names = list(dimensions.keys())


    for index, value in values.items():
        row = {}
        

        idx = int(index)
        

        temp_idx = idx
        for dim_name in reversed(dim_names):
            dim_size = len(dimensions[dim_name]['category']['index'])
            dim_idx = temp_idx % dim_size
            temp_idx = temp_idx // dim_size
            
    
            dim_keys = list(dimensions[dim_name]['category']['index'].keys())
            category_key = dim_keys[dim_idx]
            
        
            if 'label' in dimensions[dim_name]['category']:
                row[dim_name] = dimensions[dim_name]['category']['label'].get(category_key, category_key)
            else:
                row[dim_name] = category_key
        
    
        row['value'] = value
        rows.append(row)


    df = pd.DataFrame(rows)
    asylum_drop = df.drop(["unit","freq"], axis =1 )
    asylum_drop = asylum_drop[asylum_drop["value"] != 0]
    asylum_drop = asylum_drop[(asylum_drop["age"] != "Total") & (asylum_drop["sex"] != "Total") & (asylum_drop["decision"] != "Total") & (asylum_drop["geo"] != "European Union - 27 countries (from 2020)")]
    return asylum_drop


In [5]:
y2023_df = api_func("2023")
y2024_df = api_func("2024")
y2022_df = api_func("2022")
y2021_df = api_func("2021")

In [7]:
combined_df = pd.concat([y2021_df, y2022_df, y2023_df, y2024_df],ignore_index=True)
combined_df.head()

,time,geo,decision,age,sex,citizen,value
0,2021,Belgium,Rejected,From 14 to 17 years,Females,Afghanistan,5
1,2021,Germany,Rejected,From 14 to 17 years,Females,Afghanistan,40
2,2021,Greece,Rejected,From 14 to 17 years,Females,Afghanistan,70
3,2021,France,Rejected,From 14 to 17 years,Females,Afghanistan,15
4,2021,Croatia,Rejected,From 14 to 17 years,Females,Afghanistan,5


In [8]:
grouping = combined_df.groupby(["geo","citizen"])

In [9]:
grouping_age_decision =  combined_df.groupby(["age","decision"])
grouping_age_decision.describe()

value                           \
                                               count        mean          std   
age                 decision                                                    
65 years or over    Rejected                   860.0   21.406977    55.142398   
                    Total positive decisions   740.0   35.344595    97.510536   
From 14 to 17 years Rejected                  2074.0   45.298939   170.513278   
                    Total positive decisions  2072.0   93.629344   417.221708   
From 18 to 34 years Rejected                  6351.0  233.579751  1586.167549   
                    Total positive decisions  4411.0  270.054409  1849.029367   
From 35 to 64 years Rejected                  5582.0  124.973128   771.827258   
                    Total positive decisions  3551.0  135.709659   693.501853   
Less than 14 years  Rejected                  4058.0  104.507146   631.300096   
                    Total positive decisions  3831.0  190.133125  1190.021187   
Less than 18 years  Rejected                  4436.0  117.167493   710.601764   
                    Total positive decisions  4217.0  219.145127  1387.966060   
Unknown             Rejected                    19.0   11.842105    11.572300   
                    Total positive decisions    42.0   13.809524    16.885898   

                                                                             
                                              min  25%   50%   75%      max  
age                 decision                                                 
65 years or over    Rejected                  5.0  5.0   5.0  15.0    570.0  
                    Total positive decisions  5.0  5.0   5.0  20.0    890.0  
From 14 to 17 years Rejected                  5.0  5.0  10.0  20.0   2995.0  
                    Total positive decisions  5.0  5.0  10.0  35.0   8635.0  
From 18 to 34 years Rejected                  5.0  5.0  15.0  60.0  42160.0  
                    Total positive decisions  5.0  5.0  15.0  50.0  55690.0  
From 35 to 64 years Rejected                  5.0  5.0  10.0  45.0  19365.0  
                    Total positive decisions  5.0  5.0  10.0  40.0  14495.0  
Less than 14 years  Rejected                  5.0  5.0  10.0  40.0  14825.0  
                    Total positive decisions  5.0  5.0  10.0  45.0  28220.0  
Less than 18 years  Rejected                  5.0  5.0  10.0  40.0  17825.0  
                    Total positive decisions  5.0  5.0  10.0  50.0  34230.0  
Unknown             Rejected                  5.0  5.0  10.0  10.0     45.0  
                    Total positive decisions  5.0  5.0   5.0  10.0     80.0

In [10]:
Counter(combined_df['sex'])

Counter({'Males': 22824, 'Females': 19332, 'Unknown': 88})

In [11]:
Counter(combined_df['age'])

Counter({'From 18 to 34 years': 10762,
         'From 35 to 64 years': 9133,
         'Less than 18 years': 8653,
         'Less than 14 years': 7889,
         'From 14 to 17 years': 4146,
         '65 years or over': 1600,
         'Unknown': 61})

In [12]:
Counter(combined_df['decision'])

Counter({'Rejected': 23380, 'Total positive decisions': 18864})

In [13]:
Counter(combined_df['citizen'])

Counter({'Total': 2683,
         'Syria': 1598,
         'Afghanistan': 1537,
         'Iraq': 1232,
         'Russia': 1174,
         'Iran': 1083,
         'Somalia': 1069,
         'Türkiye': 1066,
         'Nigeria': 847,
         'Pakistan': 840,
         'Eritrea': 831,
         'Georgia': 827,
         'Venezuela': 779,
         'Ukraine': 768,
         'Cameroon': 686,
         'Democratic Republic of the Congo': 672,
         'Colombia': 667,
         'Morocco': 642,
         'Stateless': 628,
         'Egypt': 626,
         'Palestine*': 624,
         'Ethiopia': 569,
         'Albania': 561,
         'Sudan': 551,
         'Yemen': 544,
         'Guinea': 537,
         'Algeria': 520,
         'Belarus': 516,
         'China': 505,
         'Armenia': 504,
         'Unknown': 480,
         'Lebanon': 468,
         'El Salvador': 429,
         'Sri Lanka': 426,
         'Côte d’Ivoire': 425,
         'Bangladesh': 421,
         'Tunisia': 419,
         'Azerbaijan': 407,
    

In [14]:
Counter(combined_df['geo'])

Counter({'France': 6270,
         'Germany': 4901,
         'Italy': 3506,
         'Belgium': 3286,
         'Sweden': 3097,
         'Spain': 3092,
         'Netherlands': 2239,
         'Greece': 2166,
         'Switzerland': 1968,
         'Austria': 1836,
         'Ireland': 1221,
         'Cyprus': 1107,
         'Finland': 880,
         'Poland': 744,
         'Romania': 717,
         'Denmark': 628,
         'Norway': 625,
         'Luxembourg': 502,
         'Czechia': 464,
         'Iceland': 450,
         'Bulgaria': 422,
         'Lithuania': 365,
         'Malta': 348,
         'Portugal': 276,
         'Latvia': 247,
         'Croatia': 191,
         'Estonia': 176,
         'Montenegro': 174,
         'Slovenia': 160,
         'Slovakia': 124,
         'Hungary': 55,
         'Liechtenstein': 7})

In [24]:
fig = px.histogram(combined_df,
            x='geo',
            color='decision',
            barmode='group',
            labels={'geo': 'Country', 'decision': 'Decision Type'},
            title='Decisions by Country')
fig.update_layout(width=900, height=500)
fig.show()
p.offline.plot(fig, filename='decisionsByCountry.html')

'decisionsByCountry.html'